Fix a subgroup inclusion $H\leq G$. The function ``print_symbolic_norm`` displays formulas for the coefficients for the norm
$$ 
    \text{nm}_H^G \left( \sum a_i \cdot [H/K_i] \right) \in A(G)
$$
as polynomials in the coefficients $a_i$.

In [1]:
def print_symbolic_norm(H,G):
    # if H is not a subgroup of G, then throw an error 
    if not gap.IsSubgroup(G,H): 
        raise ValueError("You fool! H must be a subgroup of G") # maybe the error could be nicer

    tomG = G.TableOfMarks() # create the table of marks of G
    nG = len(G.ConjugacyClassesSubgroups()) # dimension of table of marks
    matG = matrix(ZZ, tomG.MatTom()) # table of marks as a matrix 
    repsG = [tomG.RepresentativeTom(n+1) for n in range(nG)] # reps of conjugacy classes

    tomH = H.TableOfMarks() # create the table of marks of H
    nH = len(H.ConjugacyClassesSubgroups()) # dimension of table of marks
    matH = matrix(ZZ, tomH.MatTom()) # table of marks as a matrix
    repsH = [tomH.RepresentativeTom(n+1) for n in range(nH)] # reps of conjugacy classes
    
    R = PolynomialRing(QQ, nH, "a") # polynomial ring with nH many generators
    f = vector(R, R.gens()) # vector over symbolic ring R

    marksf = f * matH # convert the vector f of coefficients into an element of the ghost ring
    nm = vector(R, [1]*nG) # empty product is 1; setup for looping
    for i in range(nG):
        K = repsG[i] # we're finding the component indexed by K
        for(g,_) in G.DoubleCosetRepsAndSizes(K,H): # only need the reps of double cosets
            HcapKg = gap.Intersection(H,K.ConjugateGroup(g)) # form the intersection of H and the conjugate of K by g
            index = next(j for j,r in enumerate(repsH) if H.IsConjugate(r, HcapKg)) # find the index of the subgroup of H that represents H \cap K^g
            nm[i] *= marksf[index] # multiply the [K] component of the norm by marksf([H \cap K^g])
    nm = matG.solve_left(nm) # convert back into a list of coefficients
    
    print("Norm from", gap.StructureDescription(H), "to", gap.StructureDescription(G), "============\n")    
    print("Indices:")
    for i in range(nH):
        print(i, gap.StructureDescription(H), "/", gap.StructureDescription(repsH[i]))
        
    print("\nCoefficients of norm:")
    for i in range(nG):
        print(gap.StructureDescription(G), "/", gap.StructureDescription(repsG[i]))
        show(nm[i])

In [2]:
A5 = gap("AlternatingGroup(5)")
A4 = gap("AlternatingGroup(4)")
C3 = gap("AlternatingGroup(3)")
e = gap("SymmetricGroup(1)")

print_symbolic_norm(e,C3)
print_symbolic_norm(C3,A4)
print_symbolic_norm(A4,A5)

Norm from 1 to C3 ============

Indices:
0 1 / 1

Coefficients of norm:
C3 / 1


1/3*a^3 - 1/3*a

C3 / C3


a

Norm from C3 to A4 ============

Indices:
0 C3 / 1
1 C3 / C3

Coefficients of norm:
A4 / 1


...

2*a1 + 3*a3

A5 / A4


a2*a4 + a4^2 - a4

A5 / A5


a4